In [1]:
import cv2
from picamera2 import Picamera2
import matplotlib.pyplot as plt
import os
import numpy as np
import time
os.environ["DISPLAY"] = ":0"

In [2]:
def fill_white_neighbors(gray_img):
    # Make a copy of the image to preserve the original
    output = gray_img.copy()
    
    # Use slicing to check the pixel above and below for every pixel (excluding the first and last rows)
    top = gray_img[:-2, :]
    bottom = gray_img[2:, :]
    
    # Create a mask where both top and bottom neighbors are white (255)
    mask = (top == 255) & (bottom == 255)
    
    # Apply the mask to the central region of the output image
    output[1:-1, :][mask] = 255

    return output

def fill_white_neighbors2(gray_img):
    # Make a copy of the image to avoid modifying the original
    output = gray_img.copy()
    
    # Define the central region (rows 2 to -2) where both top 2 and bottom 2 exist
    # This assumes that gray_img.shape[0] is the height
    # For each pixel in rows 2 through (height - 3), we want to check:
    #    top candidates: row-2 and row-1, and
    #    bottom candidates: row+1 and row+2.
    # These slices are arranged so that their first dimension aligns with the central region.
    
    # Top neighbor candidates
    top_candidate1 = gray_img[:-4, :]   # rows 0 to H-4
    top_candidate2 = gray_img[1:-3, :]   # rows 1 to H-3
    
    # Bottom neighbor candidates
    bottom_candidate1 = gray_img[3:-1, :]  # rows 3 to H-1
    bottom_candidate2 = gray_img[4:, :]    # rows 4 to H
    
    # Check if at least one of the top two pixels is white (255)
    top_white = (top_candidate1 == 255) | (top_candidate2 == 255)
    
    # Check if at least one of the bottom two pixels is white (255)
    bottom_white = (bottom_candidate1 == 255) | (bottom_candidate2 == 255)
    
    # Create a mask for the central region where both conditions are met
    mask = top_white & bottom_white
    
    # Apply the mask to the central region of the output image (rows 2 to -2)
    output[2:-2, :][mask] = 255
    
    return output

In [ ]:
import time
import cv2
import numpy as np
from picamera2 import Picamera2
import math

# Initialize and configure the PiCamera2
picam2 = Picamera2()
picam2.preview_configuration.main.size = (3280, 2464)
picam2.preview_configuration.main.format = "RGB888"
picam2.preview_configuration.align()
picam2.configure("preview")
picam2.start()
picam2.start(show_preview=False)

# Set minimum area (number of pixels) for a region to be considered valid
min_area = 40

while True:
    start_time = time.time()
    
    # Capture frame and resize it
    frame = picam2.capture_array()
    frame = cv2.resize(frame, (256, 256))
    
    # Convert frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define HSV ranges for red
    # lower_red1 = np.array([0, 150, 70])
    # upper_red1 = np.array([10, 255, 255])
    # lower_red2 = np.array([170, 150, 70])
    # upper_red2 = np.array([180, 255, 255])
    
    lower_red1 = np.array([0, 200, 0])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 200, 0])
    upper_red2 = np.array([180, 255, 255])

    # Create and combine red masks
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask1, mask2)
    
    # Refine the mask with morphological closing to fill gaps
    # Increase kernel size for closing
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, kernel)

    # Optionally, add a dilation step
    kernel_dilate = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask_red = cv2.dilate(mask_red, kernel_dilate, iterations=1)

    mask_red = fill_white_neighbors2(mask_red)
    
    # Use connected components to extract regions
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_red, connectivity=8)
    
    # Convert the mask to BGR for visualization (drawing colored boxes and markers)
    mask_red_color = cv2.cvtColor(mask_red, cv2.COLOR_GRAY2BGR)

    shelf_sides = []
    # Process each component (skip label 0 which is the background)
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area >= min_area:
            
            x = stats[i, cv2.CC_STAT_LEFT]
            y = stats[i, cv2.CC_STAT_TOP]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]

            if h/w > 5:

                shelf_sides.append((x, y, w, h))

                # Draw bounding box and centroid on the visualization image
                cv2.rectangle(mask_red_color, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    if len(shelf_sides) == 2:
        x1, y1, w1, h1 = shelf_sides[0]
        x2, y2, w2, h2 = shelf_sides[1]
        cv2.line(mask_red_color, (x1 + w1//2, y1 + h1//2), (x2 + w2//2, y2 + h2//2), (0, 0, 255), 2)

        dx = x1 + w1//2 - x2 - w2//2
        dy = y1 + h1//2 - y2 - h2//2
        angle_rad = math.atan2(-dy, dx)
        angle_deg = math.degrees(angle_rad)
        angle_deg = (angle_deg + 360) % 360

        if angle_deg > 180:
            angle_deg -= 180
        if angle_deg > 90:
            angle_deg = angle_deg - 180
        else:
            angle_deg = angle_deg

        print(f"Line angle: {angle_deg:.2f}")
        cv2.putText(mask_red_color, f"FPS: {angle_deg:.2f}", (10, 30), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Calculate and print FPS
    end_time = time.time()
    fps = 1.0 / (end_time - start_time)
    print(f"FPS: {fps:.2f}")
    
    # Display the processed image
    cv2.imshow("Connected Red Regions", mask_red_color)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

[8:00:58.979595015] [41171]  INFO Camera camera_manager.cpp:327 libcamera v0.4.0+53-29156679
[8:00:59.013986680] [41188]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[8:00:59.015838989] [41188]  WARN RPI vc4.cpp:393 Mismatch between Unicam and CamHelper for embedded data usage!
[8:00:59.016504109] [41188]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx219@10 to Unicam device /dev/media3 and ISP device /dev/media0
[8:00:59.016555681] [41188]  INFO RPI pipeline_base.cpp:1121 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[8:00:59.026689614] [41171]  INFO Camera camera.cpp:1202 configuring streams: (0) 3264x2464-RGB888 (1) 3280x2464-SBGGR10_CSI2P
[8:00:59.027146835] [41188]  INFO RPI vc4.cpp:622 Sensor: /base/soc/i2c0mux/i2c@1/imx219@10 - Selected sensor format: 3280x2464-SBGGR10_1X10 - Selected unicam format: 3280x2464-pBAA


Line angle: 0.31
FPS: 2.13
Line angle: 0.31
FPS: 12.37
Line angle: 0.31
FPS: 18.91
Line angle: 0.31
FPS: 15.41
Line angle: 0.31
FPS: 27.03
Line angle: 0.31
FPS: 19.11
Line angle: 0.31
FPS: 34.59
Line angle: 0.31
FPS: 28.47
Line angle: 0.31
FPS: 24.80
Line angle: 0.31
FPS: 26.89
Line angle: 0.31
FPS: 27.77
Line angle: 0.31
FPS: 33.03
Line angle: 0.31
FPS: 25.98
Line angle: 0.31
FPS: 24.59
Line angle: 0.31
FPS: 23.84
Line angle: 0.31
FPS: 24.19
Line angle: 0.31
FPS: 22.02
Line angle: 0.31
FPS: 24.47
Line angle: 0.31
FPS: 24.02
Line angle: 0.31
FPS: 21.61
Line angle: 0.31
FPS: 21.83
Line angle: 0.31
FPS: 22.88
Line angle: 0.31
FPS: 24.13
Line angle: 0.31
FPS: 22.90
Line angle: 0.31
FPS: 26.19
Line angle: 0.31
FPS: 23.01
Line angle: 0.31
FPS: 23.45
Line angle: 0.31
FPS: 22.31
Line angle: 0.31
FPS: 24.86
Line angle: 0.31
FPS: 23.30
Line angle: 0.31
FPS: 24.50
Line angle: 0.31
FPS: 21.45
Line angle: 0.31
FPS: 25.56
Line angle: 0.31
FPS: 23.79
Line angle: 0.31
FPS: 23.32
Line angle: 0.31
FPS:

In [ ]:
import time
import cv2
import numpy as np
from picamera2 import Picamera2

# Initialize and configure the PiCamera2
picam2 = Picamera2()
picam2.preview_configuration.main.size = (3280, 2464)
picam2.preview_configuration.main.format = "RGB888"
picam2.preview_configuration.align()
picam2.configure("preview")
picam2.start()
picam2.start(show_preview=False)
counter = 0

while True:
    start_time = time.time()
    
    # Capture frame-by-frame and resize for processing
    frame = picam2.capture_array()
    frame = cv2.resize(frame, (256, 256))
    
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define HSV ranges for red
    lower_red1 = np.array([0, 150, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 150, 70])
    upper_red2 = np.array([180, 255, 255])
    
    # Create and combine masks for both red ranges
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask1, mask2)
    
    # Apply morphological closing to refine the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, kernel)
    
    # Run Canny edge detection on the refined red mask
    edges = cv2.Canny(mask_red, 50, 150)
    
    # Run the Probabilistic Hough Line Transform on the edges
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=40,
                            minLineLength=40, maxLineGap=5)
    
    # Convert the binary red mask to BGR for visualization
    mask_red_color = cv2.cvtColor(mask_red, cv2.COLOR_GRAY2BGR)
    
    # If any lines are detected, print and draw them
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            print("Hough line:", x1, y1, x2, y2)
            cv2.line(mask_red_color, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Calculate FPS
    end_time = time.time()
    fps = 1.0 / (end_time - start_time)
    print(f"FPS: {fps:.2f}")
    
    # Display the resulting image
    cv2.imshow("Red Mask with Hough Lines", mask_red_color)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

[7:40:19.260246660] [40703]  INFO Camera camera_manager.cpp:327 libcamera v0.4.0+53-29156679
[7:40:19.297889527] [40720]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[7:40:19.299714167] [40720]  WARN RPI vc4.cpp:393 Mismatch between Unicam and CamHelper for embedded data usage!
[7:40:19.300374086] [40720]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx219@10 to Unicam device /dev/media3 and ISP device /dev/media0
[7:40:19.300423731] [40720]  INFO RPI pipeline_base.cpp:1121 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[7:40:19.310426909] [40703]  INFO Camera camera.cpp:1202 configuring streams: (0) 3264x2464-RGB888 (1) 3280x2464-SBGGR10_CSI2P
[7:40:19.310869065] [40720]  INFO RPI vc4.cpp:622 Sensor: /base/soc/i2c0mux/i2c@1/imx219@10 - Selected sensor format: 3280x2464-SBGGR10_1X10 - Selected unicam format: 3280x2464-pBAA


Hough line: 15 53 36 227
Hough line: 18 36 28 134
Hough line: 211 186 215 146
Hough line: 31 154 38 221
Hough line: 219 129 223 85
Hough line: 26 151 34 217
FPS: 2.09
Hough line: 17 73 35 225
Hough line: 17 36 29 137
Hough line: 212 177 218 108
Hough line: 33 178 39 226
Hough line: 220 107 224 60
FPS: 10.33
Hough line: 14 45 36 227
Hough line: 211 186 218 107
Hough line: 23 89 28 134
Hough line: 30 152 39 223
Hough line: 27 160 33 209
FPS: 17.96
Hough line: 15 54 36 227
Hough line: 30 145 38 220
Hough line: 211 186 217 122
Hough line: 18 36 28 134
Hough line: 27 160 33 209
FPS: 24.67
Hough line: 30 145 38 220
Hough line: 15 53 36 227
Hough line: 212 177 217 127
Hough line: 18 43 29 137
Hough line: 25 143 33 209
FPS: 26.10
Hough line: 16 64 36 227
Hough line: 17 36 28 134
Hough line: 211 186 216 134
Hough line: 30 150 36 199
FPS: 18.89
Hough line: 17 73 36 227
Hough line: 211 187 218 114
Hough line: 30 145 38 220
Hough line: 19 52 29 137
FPS: 28.96
Hough line: 15 54 36 227
Hough line: 1

In [ ]:
picam2 = Picamera2()
picam2.preview_configuration.main.size = (3280, 2464)
picam2.preview_configuration.main.format = "RGB888"
picam2.preview_configuration.align()
picam2.configure("preview")
picam2.start()
picam2.start(show_preview=False)
counter = 0

while True:
    start_time = time.time()
    
    # Capture frame-by-frame
    frame = picam2.capture_array()

    # Run YOLO inference on the frame with imgsz=256
    frame = cv2.resize(frame, (256, 256))

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Lower range for red (hue near 0)
    # lower_red1 = np.array([0, 160, 100])
    # upper_red1 = np.array([5, 255, 255])
    # lower_red2 = np.array([175, 160, 100])
    # upper_red2 = np.array([180, 255, 255])

    lower_red1 = np.array([0, 150, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 150, 70])
    upper_red2 = np.array([180, 255, 255])

    # Create masks for both red ranges
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Combine the masks
    mask_red = cv2.bitwise_or(mask1, mask2)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, kernel)
    # Find contours on the red mask
    # contours, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # # Convert the mask to a BGR image so we can draw colored circles
    mask_red_color = cv2.cvtColor(mask_red, cv2.COLOR_GRAY2BGR)

    # red_centers = []
    # min_area = 1  # Adjust this threshold as needed for your image

    # for cnt in contours:
    #     area = cv2.contourArea(cnt)
    #     if area > min_area:
    #         # Compute the centroid using image moments
    #         M = cv2.moments(cnt)
    #         if M["m00"] != 0:
    #             cX = int(M["m10"] / M["m00"])
    #             cY = int(M["m01"] / M["m00"])
    #             red_centers.append((cX, cY))
    #             # Draw a small circle at the centroid on the mask image
    #             cv2.circle(mask_red_color, (cX, cY), 2, (0, 255, 0), -1)

    # print("Centers of green clusters:", red_centers)

    # Calculate FPS
    end_time = time.time()
    fps = 1.0 / (end_time - start_time)

    # for center in red_centers:
        # cv2.circle(mask_red_color, center, 3, (0, 255, 0), -1)
    
    # Overlay the FPS on the annotated frame
    # cv2.putText(mask_red_color, f"FPS: {len(red_centers):.2f}", (10, 30), 
    #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Print FPS to the console (optional)
    print(f"FPS: {fps:.2f}")

    cv2.imshow("Red Mask", mask_red_color)
    cv2.waitKey(1)

[1:22:08.248475751] [3332]  INFO Camera camera_manager.cpp:327 libcamera v0.4.0+53-29156679
[1:22:08.281140680] [3349]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[1:22:08.282991083] [3349]  WARN RPI vc4.cpp:393 Mismatch between Unicam and CamHelper for embedded data usage!
[1:22:08.283614705] [3349]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx219@10 to Unicam device /dev/media3 and ISP device /dev/media0
[1:22:08.283658299] [3349]  INFO RPI pipeline_base.cpp:1121 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[1:22:08.293953370] [3332]  INFO Camera camera.cpp:1202 configuring streams: (0) 3264x2464-RGB888 (1) 3280x2464-SBGGR10_CSI2P
[1:22:08.294391525] [3349]  INFO RPI vc4.cpp:622 Sensor: /base/soc/i2c0mux/i2c@1/imx219@10 - Selected sensor format: 3280x2464-SBGGR10_1X10 - Selected unicam format: 3280x2464-pBAA


Centers of green clusters: [(24, 133)]
FPS: 1.88
Centers of green clusters: [(24, 133)]
FPS: 11.40
Centers of green clusters: [(24, 135)]
FPS: 17.36
Centers of green clusters: [(24, 133)]
FPS: 22.24
Centers of green clusters: [(24, 134)]
FPS: 22.21
Centers of green clusters: [(24, 133)]
FPS: 35.02
Centers of green clusters: [(24, 133)]
FPS: 16.59
Centers of green clusters: [(24, 133)]
FPS: 16.35
Centers of green clusters: [(24, 134)]
FPS: 15.83
Centers of green clusters: [(24, 133)]
FPS: 15.79
Centers of green clusters: [(24, 134)]
FPS: 15.95
Centers of green clusters: [(24, 134)]
FPS: 15.92
Centers of green clusters: [(24, 133)]
FPS: 15.95
Centers of green clusters: [(24, 133)]
FPS: 15.85
Centers of green clusters: [(24, 133)]
FPS: 15.58
Centers of green clusters: [(24, 133)]
FPS: 15.81
Centers of green clusters: [(24, 134)]
FPS: 15.95
Centers of green clusters: [(24, 134)]
FPS: 16.19
Centers of green clusters: [(24, 133)]
FPS: 16.25
Centers of green clusters: [(24, 133)]
FPS: 16.01
C